In [19]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.LSTM import LSTM
from motion_generation.MotionGeneration import MotionGenerationModel
# from motion_generation.MotionGeneration_v2 import MotionGenerationModel as MotionGenerationModel_v2
# from motion_generation.MotionGeneration_v3 import MotionGenerationModel as MotionGenerationModel_v3
# from motion_generation.MotionGenerationRNN import MotionGenerationModelRNN
# from motion_generation.MotionGenerationBatch import MotionGenerationModelBatch
from rig_agnostic_encoding.models.MLP import MLP
# from rig_agnostic_encoding.models.MLP_MIX import MLP_MIX
from rig_agnostic_encoding.models.VAE import VAE
from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
# from DataProcessingFunctions import calc_tta_embedding
from GlobalSettings import MODEL_PATH
import bz2

import torch
import torch.nn as nn
from cytoolz import sliding_window, accumulate
from operator import add

import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib

In [2]:
class MLP_MIX(pl.LightningModule):
    def __init__(self, config:dict=None, input_dims:list=None, pose_labels=None,
                 train_set=None, val_set=None, test_set=None,
                 name:str="model", save_period=5, workers=6):

        super().__init__()

        M = len(input_dims)

        self.name = name
        self.input_dims = input_dims
        self.input_slice = [0] + list(accumulate(add, input_dims))

        self.act = nn.ELU
        self.save_period = save_period
        self.workers = workers
        self.pose_labels = pose_labels if pose_labels is not None else [None for _ in range(M)]

        self.config = config
        self.hidden_dim = config["hidden_dim"]
        self.keep_prob = config["keep_prob"]
        self.k = config["k"]
        self.learning_rate = config["lr"]
        self.batch_size = config["batch_size"]

        self.loss_fn = config["loss_fn"] if "loss_fn" in config else nn.functional.mse_loss
        self.opt = config["optimizer"] if "optimizer" in config else torch.optim.Adam
        self.scheduler = config["scheduler"] if "scheduler" in config else None
        self.scheduler_param = config["scheduler_param"] if "scheduler_param" in config else None

        self.models = []
        self.active_models = []
        self.cluster_model = nn.Sequential()

        self.train_set = train_set
        self.val_set = val_set
        self.test_set = test_set

        self.best_val_loss = np.inf

        self.models = [MLP(config=self.config, dimensions=[self.input_dims[i]], pose_labels=self.pose_labels[i],
                           name="M"+str(i), single_module=0) for i in range(M)]
        self.active_models = self.models

        self.cluster_model = nn.Sequential(
            nn.Linear(in_features=self.k, out_features=self.k))
        self.init_params(self.cluster_model)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x_tensors = [x[:, d0:d1] for d0, d1 in zip(self.input_slice[:-1], self.input_slice[1:])]

        encoded = [m.encode(x_tensors[i]) for i, m in enumerate(self.active_models)]
        embeddings = [self.cluster_model(vec) for vec in encoded]
        decoded = [m.decode(embeddings[i]) for i, m in enumerate(self.active_models)]

        return torch.cat(decoded, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log("ptl/train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log('ptl/val_loss', loss, prog_bar=True)
        return {"val_loss":loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, x.shape[-1])
        y = y.view(-1, y.shape[-1])
        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log('ptl/test_loss', loss, prog_bar=True)
        return {"test_loss":loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            if self.current_epoch % self.save_period == 0:
                self.save_checkpoint(best_val_loss=self.best_val_loss.item())

    def save_checkpoint(self, best_val_loss:float=np.inf, checkpoint_dir=MODEL_PATH):
        path = os.path.join(checkpoint_dir, self.name)

        config = {
            "hidden_dim": self.hidden_dim,
            "k": self.k,
            "lr": self.learning_rate,
            "batch_size": self.batch_size,
            "keep_prob":self.keep_prob,
            "optimizer": self.opt,
            "scheduler": self.scheduler,
            "scheduler_param": self.scheduler_param,
        }

        model_paths = [m.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path) for m in self.models]
        model = {"config":config, "name":self.name, "model_paths":model_paths,
                 "input_dims":self.input_dims, "pose_labels":self.pose_labels,
                 "cluster_model":self.cluster_model.state_dict()
                 }

        if not os.path.exists(checkpoint_dir):
            os.mkdir(checkpoint_dir)
        path = os.path.join(checkpoint_dir, self.name)
        if not os.path.exists(path):
            os.mkdir(path)

        filePath = os.path.join(path, str(best_val_loss)+"."+str(self.k)+".pbz2")
        with bz2.BZ2File(filePath, "w") as f:
            pickle.dump(model, f)
        return filePath

    @staticmethod
    def load_checkpoint(filePath):
        with bz2.BZ2File(filePath, "rb") as f:
            obj = pickle.load(f)
        model = MLP_MIX(config=obj["config"], name=obj["name"],
                    input_dims=obj["input_dims"], pose_labels=obj["pose_labels"])

        models = [MLP.load_checkpoint(path) for path in obj["model_paths"]]
        model.models = models
        model.cluster_model.load_state_dict(obj["cluster_model"])
        return model

    def configure_optimizers(self):
        optimizer = self.opt(self.parameters(), lr=self.learning_rate)
        if self.scheduler is not None:
            scheduler = self.scheduler(optimizer, **self.scheduler_param)
            return [optimizer], [scheduler]
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(.01)

    def add_models(self, input_dims:list=None, pose_labels:list=None, freeze=False):
        n = len(self.models)+1
        if pose_labels is not None:
            self.models += [MLP(config=self.config, dimensions=[input_dims[i]], pose_labels=pose_labels[i],
                            name="M" + str(i+n), single_module=0) for i in range(len(input_dims))]
        else:
            self.models += [MLP(config=self.config, dimensions=[input_dims[i]],
                            name="M" + str(i+n), single_module=0) for i in range(len(input_dims))]

        if freeze:
            for model in self.active_models:
                model.freeze(True)
            self.active_models = self.models[n-1:]
            self.input_dims = input_dims
        else:
            self.active_models = self.models
            self.input_dims += input_dims

        self.input_slice = [0] + list(accumulate(add, self.input_dims))


In [54]:
def calc_tta_embedding(embedd_dim:float, tta, basis: float = 1e5, window=30, device="cuda"):
    """
    Calculates time-to-arriaval-embeddings, according to the paper [Robust Motion In-betweening]
    :param contacts:
    :param basis
    :return:
    """

    # if embedd_dim % 2 != 0:
    #     d1 = embedd_dim / 2 + 1
    #     d2 = embedd_dim / 2
    # else:
    #     d1 = d2 = embedd_dim / 2

    basis = torch.Tensor([basis]).to(device)

    tta[tta > window] = 0
    d1 = torch.arange(0, embedd_dim, 2).to(device)
    d2 = torch.arange(1, embedd_dim, 2).to(device)

    z_sin = torch.sin(tta / (torch.pow(basis, 2.0*d1 / embedd_dim)))
    z_cos = torch.cos(tta / (torch.pow(basis, 2.0*d2 / embedd_dim)))

    z = torch.zeros(embedd_dim).to(device)
    z[0::2] += z_sin
    z[1::2] += z_cos
    return z

class MotionGenerationModel(pl.LightningModule):
    def __init__(self, config:dict=None, Model=None, pose_autoencoder=None, middle_layer=None, feature_dims=None,
                 input_slicers:list=None, output_slicers:list=None,
                 train_set=None, val_set=None, test_set=None, name="MotionGeneration", workers=8):
        super().__init__()

        self.feature_dims = feature_dims
        self.config=config
        self.workers = workers

        self.loss_fn = config["loss_fn"] if "loss_fn" in config else nn.functional.mse_loss
        self.opt = config["optimizer"] if "optimizer" in config else torch.optim.Adam
        self.scheduler = config["scheduler"] if "scheduler" in config else None
        self.scheduler_param = config["scheduler_param"] if "scheduler_param" in config else None
        self.batch_size = config["batch_size"]
        self.learning_rate = config["lr"]
        self.seq_len = config["seq_len"]

        self.autoregress_prob = config["autoregress_prob"]
        self.autoregress_inc = config["autoregress_inc"]
        self.autoregress_ep = config["autoregress_ep"]
        self.autoregress_max_prob = config["autoregress_max_prob"]

        self.best_val_loss = np.inf
        self.phase_smooth_factor = 0.9

        self.pose_autoencoder = pose_autoencoder if pose_autoencoder is not None else \
            MLP(config=config, dimensions=[feature_dims["pose_dim"]], name="PoseAE")
        self.middle_layer = middle_layer if middle_layer is not None else \
            nn.Linear(in_features=self.pose_autoencoder.dimensions[-1], out_features=self.pose_autoencoder.dimensions[-1])
        self.use_label = pose_autoencoder is not None and pose_autoencoder.use_label

        cost_hidden_dim = config["cost_hidden_dim"]
        self.cost_encoder = MLP(config=config,
                            dimensions=[feature_dims["cost_dim"], cost_hidden_dim, cost_hidden_dim, cost_hidden_dim],
                            name="CostEncoder", single_module=-1)

        self.generationModel =  Model(config=config,
                                      dimensions=[feature_dims["g_input_dim"], feature_dims["g_output_dim"]],
                                      phase_input_dim = feature_dims["phase_dim"])

        self.input_dims = input_slicers
        self.output_dims = output_slicers
        self.in_slices = [0] + list(accumulate(add, input_slicers))
        self.out_slices = [0] + list(accumulate(add, output_slicers))
        self.noise = torch.autograd.Variable(torch.zeros(1)).to(config["device"])
        self.std = 1

        self.train_set = train_set
        self.val_set = val_set
        self.test_set = test_set
        self.name = name

        self.automatic_optimization = False

    def forward(self, x):
        x_tensors = [x[:, d0:d1] for d0, d1 in zip(self.in_slices[:-1], self.in_slices[1:])]

        pose_h = self.middle_layer(self.pose_autoencoder.encode(x_tensors[1]))
        embedding = torch.cat([pose_h, self.cost_encoder(x_tensors[2])], dim=1)

        # tta = torch.max(x_tensors[-1])
        # tta_embedding = calc_tta_embedding(embedd_dim=embedding.shape[-1], tta=tta, device=self.config["device"])
        # embedding += tta
        # self.std = min(1, torch.mean(x_tensors[-1])/30.0)
        self.noise.data.normal_(0, std=self.std)
        embedding += self.noise

        out = self.generationModel(embedding, x_tensors[0])
        out_tensors = [out[:, d0:d1] for d0, d1 in zip(self.out_slices[:-1], self.out_slices[1:])]
        phase = self.update_phase(x_tensors[0], out_tensors[0])
        new_pose = self.pose_autoencoder.decode(out_tensors[1])

        return [phase, new_pose, out_tensors[-1]]



    def step(self, x, y):
        opt = self.optimizers()

        n = x.size()[1]
        tot_loss = 0

        x_c = x[:,0,:]
        autoregress_bools = torch.randn(n) < self.autoregress_prob
        for i in range(1, n):
            y_c = y[:,i-1,:]

            out= self(x_c)
            recon = torch.cat(out, dim=1)
            loss = self.loss_fn(recon, y_c)
            tot_loss += loss.detach()

            opt.zero_grad()
            self.manual_backward(loss)
            opt.step()

            if autoregress_bools[i]:
                x_c = torch.cat(out,dim=1).detach()
            else:
                x_c = x[:,i,:]

        tot_loss /= float(n)
        return tot_loss

    def step_eval(self, x, y):
        n = x.size()[1]
        tot_loss = 0

        x_c = x[:,0,:]
        autoregress_bools = torch.randn(n) < self.autoregress_prob
        for i in range(1, n):
            y_c = y[:,i-1,:]

            out= self(x_c)
            recon = torch.cat(out, dim=1)
            loss = self.loss_fn(recon, y_c)
            tot_loss += loss.detach()

            if autoregress_bools[i]:
                x_c = torch.cat(out,dim=1).detach()
            else:
                x_c = x[:,i,:]

        tot_loss /= float(n)
        return tot_loss

    def training_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, self.seq_len, x.shape[-1])
        y = y.view(-1, self.seq_len, y.shape[-1])
        loss = self.step(x,y)
        self.log("ptl/train_loss", loss, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, self.seq_len, x.shape[-1])
        y = y.view(-1, self.seq_len, y.shape[-1])
        loss = self.step_eval(x,y)
        self.log("ptl/val_loss", loss, prog_bar=True)

        return {"val_loss":loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, self.seq_len, x.shape[-1])
        y = y.view(-1, self.seq_len, y.shape[-1])
        loss = self.step_eval(x,y)
        self.log("ptl/test_loss", loss, prog_bar=True)

        return {"test_loss":loss}

    def validation_epoch_end(self, outputs):
        if self.current_epoch > 0 and self.current_epoch % self.autoregress_ep == 0:
            self.autoregress_prob = min(self.autoregress_max_prob, self.autoregress_prob+self.autoregress_inc)

        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            self.save_checkpoint(best_val_loss=self.best_val_loss.item())

    def save_checkpoint(self, best_val_loss=np.inf, checkpoint_dir=MODEL_PATH):
        path = os.path.join(checkpoint_dir, self.name)

        config = {}
        for key, val in self.config.items():
            config[key] = val

        pose_autoencoder_path = self.pose_autoencoder.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path)
        cost_encoder_path = self.cost_encoder.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path)
        generationModel_path = self.generationModel.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path)

        model = {"name":self.name,
                 "config":config,
                 "pose_autoencoder_path":pose_autoencoder_path,
                 "cost_encoder_path": cost_encoder_path,
                 "motionGenerationModelPath":generationModel_path,
                 "in_slices":self.in_slices,
                 "out_slices":self.out_slices,
                 "feature_dims":self.feature_dims,
                 "middle_layer_dict":self.middle_layer.state_dict(),
                 }

        if not os.path.exists(path):
            os.mkdir(path)
        with bz2.BZ2File(os.path.join(path,
                                      str(best_val_loss)+".pbz2"), "w") as f:
            pickle.dump(model, f)

    @staticmethod
    def load_checkpoint(filename, Model, MiddleModel:nn.Module=None):
        with bz2.BZ2File(filename, "rb") as f:
            obj = pickle.load(f)

        pose_autoencoder = MLP.load_checkpoint(obj["pose_autoencoder_path"])
        cost_encoder = MLP.load_checkpoint(obj["cost_encoder_path"])
        generationModel = Model.load_checkpoint(obj["motionGenerationModelPath"])

        model = MotionGenerationModel(config=obj["config"], feature_dims=obj["feature_dims"], Model=Model,
                                      input_slicers=obj["in_slices"], output_slicers=obj["out_slices"],
                                      name=obj["name"])
        if MiddleModel is None:
            MiddleModel = nn.Linear(in_features=pose_autoencoder.dimensions[-1], out_features=pose_autoencoder.dimensions[-11])

        MiddleModel.load_state_dict(obj["middle_layer_dict"])
        model.in_slices = obj["in_slices"]
        model.out_slices = obj["out_slices"]
        model.pose_autoencoder = pose_autoencoder
        model.cost_encoder = cost_encoder
        model.generationModel = generationModel

        return model

    def update_phase(self, p1, p2):
        return self.phase_smooth_factor * p2 + (1-self.phase_smooth_factor)*p1

    def configure_optimizers(self):
        optimizer = self.opt(self.parameters(), lr=self.learning_rate)
        if self.scheduler is not None:
            scheduler = self.scheduler(optimizer, **self.scheduler_param)
            return [optimizer], [scheduler]
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def swap_pose_encoder(self, pose_encoder=None,
                          input_dim=None, output_dim=None,
                          feature_dims=None, freeze=False):
        self.pose_autoencoder = pose_encoder
        self.input_dims = input_dim
        self.output_dims = output_dim
        self.feature_dims = feature_dims
        self.in_slices = [0] + list(accumulate(add, self.input_dims))
        self.out_slices = [0] + list(accumulate(add, self.output_dims))

        if freeze:
            self.generationModel.freeze()
            self.cost_encoder.freeze()


In [34]:
config = {
    "hidden_dim": 512,
    "k": 256,
    "lr": 1e-4,
    "batch_size": 32,
    "keep_prob": .2,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.AdamW,
    # "scheduler":torch.optim.lr_scheduler.StepLR,
    # "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":128,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0,
    "autoregress_inc":.2,
    "autoregress_ep":50,
    "autoregress_max_prob":.5,
    "cost_hidden_dim":128,
    "seq_len":13,
    "device":"cuda"
    }

In [6]:
MAX_FILES = -1
data_path = "/home/nuoc/Documents/MEX/data/Dataset_R1_One_1"
dir = []
file_paths = []
for dname, dirs, files in os.walk(data_path):
    for i, file in enumerate(files):
        file_paths.append(os.path.join(dname, file))
        if MAX_FILES > 0 and i >= MAX_FILES:
            break




In [7]:
phase_features = ["phase_vec_l2", "targetPosition", "targetRotation"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost", "tta"]
features = phase_features + pose_features + cost_features
clips = []
feature_dims = {}

In [8]:
data = F.process_data_multithread(file_paths, features)

feature_dims = data[0][1]

clips = [np.copy(d[0]) for d in data]


2021-05-07 11:48:58,663	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [11]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
print(phase_dim, " ", pose_dim, " ", cost_dim)

56   372   28


In [12]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips])
y_tensors = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips])

dataset = TensorDataset(torch.Tensor(x_tensors), torch.Tensor(y_tensors))
N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio
train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
print(len(train_set), len(val_set), len(test_set))

168 36 36


In [16]:
dataset_name = "R1_MoGenData_ALL"

In [17]:
data = {"data":[train_set, val_set, test_set],
        "feature_dims":feature_dims,
        "dims":[phase_dim, pose_dim, cost_dim]}
F.save(data, dataset_name, "/home/nuoc/Documents/MEX/data")

In [14]:
# obj = F.load("/home/nuoc/Documents/MEX/data/"+dataset_name+".pbz2")
# train_set, val_set, test_set = obj["data"]
# feature_dims = obj["feature_dims"]
# phase_dim, pose_dim, cost_dim= obj["dims"]

obj = F.load("/home/nuoc/Documents/MEX/data/"+dataset_name2+".pbz2")
train_set2, val_set2, test_set2 = obj["data"]
feature_dims2 = obj["feature_dims"]
phase_dim2, pose_dim2, cost_dim2= obj["dims"]

In [19]:
print(len(train_set), train_set[0][0].shape)
print(len(val_set), val_set[0][0].shape)
print(len(test_set), test_set[0][0].shape)
print(feature_dims)


1362 torch.Size([299, 404])
292 torch.Size([299, 404])
293 torch.Size([299, 404])
{'phase_vec_l2': 8, 'pos': 93, 'rotMat2': 186, 'velocity': 93, 'posCost': 12, 'rotCost': 12}


In [55]:
model_name = "MLP_MoE_R1_BEST"

feature_dims2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim-feature_dims["tta"],
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim
    }

in_slice = [phase_dim, pose_dim, cost_dim-feature_dims["tta"], feature_dims["tta"]]
out_slice = [phase_dim, config["k"], cost_dim]

MLPMIX = MLP_MIX(config=config, input_dims=[pose_dim])
pose_encoder = MLPMIX.active_models[0]
middle_layer = MLPMIX.cluster_model
# cost_encoder = model2.cost_encoder

# generation_model = model2.generationModel
model = MotionGenerationModel(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=feature_dims2,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=train_set, val_set=val_set, test_set=test_set,
                                 name=model_name
                                   )
# model.generationModel = generation_model
# model.cost_encoder = cost_encoder
#
# model.cost_encoder.freeze()
# model.generationModel.freeze()
# model.middle_layer.requires_grad_(False)



In [56]:
MAX_EPOCHS = 300

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=20)
logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.1")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [57]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 1.2 M 
1 | middle_layer     | Sequential | 65.8 K
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.6 M 
------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.333    Total estimated model params size (MB)


1

In [23]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.023196307942271233, 'test_loss': 0.0217161662876606}
--------------------------------------------------------------------------------


[{'test_loss': 0.0217161662876606, 'ptl/test_loss': 0.023196307942271233}]

In [24]:
clean_checkpoints(path=os.path.join(MODEL_PATH,model_name))
# clean_checkpoints(path="/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_One_1_Full_v3_smooth_loss")

In [61]:
filename = "/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_One_1_Full_TUNE/0.06325527280569077.pbz2"
with bz2.BZ2File(filename, "rb") as f:
    obj = pickle.load(f)

pose_autoencoder = MLP.load_checkpoint(obj["pose_autoencoder_path"])
cost_encoder = MLP.load_checkpoint(obj["cost_encoder_path"])
generationModel = MoE.load_checkpoint(obj["motionGenerationModelPath"])

model = MotionGenerationModel_v2(config=obj["config"], feature_dims=obj["feature_dims"], Model=MoE,
                              input_slicers=obj["in_slices"], output_slicers=obj["out_slices"],
                              name=obj["name"])

model.in_slices = obj["in_slices"]
model.out_slices = obj["out_slices"]
model.pose_autoencoder = pose_autoencoder
model.cost_encoder = cost_encoder
model.generationModel = generationModel

In [64]:
model.test_set = test_set

[0, 62, 434, 458]
MLP(
  (encoder): Sequential(
    (fc0): Linear(in_features=372, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Sequential(
    (fc0): Linear(in_features=256, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=372, bias=True)
  )
)
MLP(
  (encoder): Sequential(
    (fc0): Linear(in_features=24, out_features=128, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=128, out_features=128, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=128, out_features=128, bias=True)
  )
  (decoder): Sequent

In [86]:
model.autoregress_prob = 0
trainer.test(model)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.03506697714328766, 'test_loss': 0.05357477068901062}
--------------------------------------------------------------------------------


[{'test_loss': 0.05357477068901062, 'ptl/test_loss': 0.03506697714328766}]

In [87]:
model.autoregress_prob = 1
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.0814722552895546, 'test_loss': 0.1374703049659729}
--------------------------------------------------------------------------------


[{'test_loss': 0.1374703049659729, 'ptl/test_loss': 0.0814722552895546}]

In [25]:
# model.autoregress_prob = 0

n = 10
idx = np.random.randint(0, len(test_set), n)
original = []
generated = []
# pose_idx_upper = feature_dims2["phase_dim"] + feature_dims["pos"] + feature_dims["rotMat2"]
pose_idx_upper = model.in_slices[1] + feature_dims["pos"] + feature_dims["rotMat2"]
print(pose_idx_upper)


287


In [35]:
model.autoregress_prob = 1
with torch.no_grad():
    model.eval()
    model.cpu()
    # for i in range(1):
    # original =
    # x = x_tensors[idx]
    x = torch.stack([test_set[i][0] for i in idx])
    y = torch.stack([test_set[i][1] for i in idx])
    shape = x.shape
    # x = x.view(-1, shape[-1])

    x = x.view(-1, 13, shape[-1])
    x_c = x[:,0,:]
    # n = shape[1]
    g_frames = []
    #
    for i in range(0, 13):
        out= model(x_c)
        x_c = torch.cat(out,dim=1).detach()
        g_frames.append(x_c.unsqueeze(1))

    # out = torch.cat(model(x), dim=1).view(shape)
            # x_c = torch.cat(out, dim=1)
    # g_frames.append(x_c.unsqueeze(1))
        # original.append(o_frames)
    # generated.append(torch.cat(g_frames, dim=1))
    # generated = out
    generated = torch.cat(g_frames, dim=1)
    generated = generated.view(shape)


In [36]:
print(y.size())
# generated = generated.view(-1, 298, generated.shape[-1])
print(generated.size())

torch.Size([10, 299, 404])
torch.Size([10, 299, 404])


In [37]:
phase_dim = feature_dims2["phase_dim"]
toPosDim = phase_dim+feature_dims["pos"]
toRotDim = toPosDim + feature_dims["rotMat2"]

gPos = generated[:, :, phase_dim:toPosDim]
gRot = generated[:, :, toPosDim:toRotDim]

oPos = y[:, :, phase_dim:toPosDim]
oRot = y[:, :, toPosDim:toRotDim]

print(gPos.shape, gRot.shape)
print(oPos.shape, oRot.shape)


torch.Size([10, 299, 93]) torch.Size([10, 299, 186])
torch.Size([10, 299, 93]) torch.Size([10, 299, 186])


In [38]:
n = 10
clip_length = gPos.shape[1]
gPos_r = gPos.reshape((n, clip_length, -1, 3))
gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
oPos_r = oPos.reshape((n, clip_length, -1, 3))
oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))

print("Pos loss: ", torch.nn.functional.mse_loss(gPos_r, oPos_r))
print("Rot loss: ", torch.nn.functional.mse_loss(gRot_r, oRot_r))

Pos loss:  tensor(0.0153)
Rot loss:  tensor(0.0500)


In [30]:
template = js.load(open("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json"))

In [39]:
def insert_pos(positions, rotations, name="Replay"):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            for j in range(shape[2]):
                template["frames"][f]["joints"][j]["position"]["x"] = positions[c,f,j,0].item()
                template["frames"][f]["joints"][j]["position"]["y"] = positions[c,f,j,1].item()
                template["frames"][f]["joints"][j]["position"]["z"] = positions[c,f,j,2].item()

                for r, cell in enumerate(["x", "y", "z"]):
                    for col, column in enumerate(["c0", "c1"]):
                        template["frames"][f]["joints"][j]["rotMat"][column][cell] = rotations[c,f,j,r, col].item()
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

# os.mkdir("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_ALL")
insert_pos(oPos_r, oRot_r, "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_ALL/Original")
insert_pos(gPos_r, gRot_r, "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_ALL/Generated")
